<a href="https://colab.research.google.com/github/seunghyeokkim/ByeolDaJul/blob/main/TF_IDF%2CLSTM_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 244.6 kB/s eta 0:00:00


In [2]:
!gdown /content/sae4k_v2.txt #파일 다운

Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=/content/sae4k_v2.txt

but Gdown can't. Please check connections and permissions.


TF-IDF

In [3]:
# 파일 내용 열고 읽기
with open("/content/sae4k_v2.txt", "r", encoding="utf-8") as file:
    content = file.read()

# 파일 보기
content_preview = content[:]
content_preview


FileNotFoundError: [Errno 2] No such file or directory: '/content/sae4k_v2.txt'

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

okt = Okt()

def extract_important_words(sentence: str, n=3) -> str:
    """
    Okt와 TF-IDF를 사용하여 문장에서 n개의 가장 중요한 단어를 추출합니다.
    """
    # Okt를 사용하여 문장 토큰화
    words = okt.nouns(sentence)  # 간단하게 하기 위해 명사만 추출합니다.

    # 단어가 n개 이하이면 모든 단어를 반환
    if len(words) <= n:
        return ' '.join(words)

    # 그렇지 않으면 각 단어의 TF-IDF 점수를 계산하고 상위 n개의 단어를 선택
    vectorizer = TfidfVectorizer().fit(words)
    word_tfidf_scores = vectorizer.transform(words).sum(axis=0).tolist()[0]

    sorted_words = [word for _, word in sorted(zip(word_tfidf_scores, words), reverse=True)]

    return ' '.join(sorted_words[:n])

# 함수 테스트
sample_sentence = "지금 너희 집으로 가고있는데 몇시쯤 도착해?"
important_words = extract_important_words(sample_sentence, 3)
print(important_words)


# Lstm모델을 이용한 모델 만들기

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# 1. 데이터 로딩
def load_and_parse_data(filepath):
    data = {
        "label": [],
        "intent_pair1": [],
        "intent_pair2": []
    }

    with open(filepath, "r", encoding="utf-8") as file:
        for line in file:
            parts = line.strip().split("\t")
            data["label"].append(int(parts[0]))
            data["intent_pair1"].append(parts[1])
            data["intent_pair2"].append(parts[2])

    return pd.DataFrame(data)

df = load_and_parse_data("/content/sae4k_v2.txt")

# 2. 데이터 전처리
MAX_WORDS = 10000 # 고려할 최대 단어수 = 최빈값 10000개만 고려하기

tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='<OOV>') #케라스 제공 텍스트 토큰화, ov_token='<OOV>' =  본적없는 단어 나올시 대체(일반화)
tokenizer.fit_on_texts(df['intent_pair1']) #텍스트 토큰화 진행

sequences = tokenizer.texts_to_sequences(df['intent_pair1']) # 텍스트 데이터를 숫자 시퀀스로 변환
X = pad_sequences(sequences)
y = to_categorical(df['label'])

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42) #분할 비율 (70% 학습, 15% 검증, 15% 테스트)더 많은 데이터 80 10 10도 가능
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 3. 모델 구성 및 학습
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(MAX_WORDS, 128, input_length=X.shape[1]), #Embedding 층: 주어진 단어의 정수 인덱스를 벡터 형태로 변환
    tf.keras.layers.LSTM(128, return_sequences=True), #LSTM 층 (128 뉴런) True 옵션을 사용해서 다음층으로 전달
    tf.keras.layers.LSTM(64), #추가처리 64층
    tf.keras.layers.Dense(64, activation='relu'), #Dense 층 (64 뉴런): 히든 레이어로, ReLU 활성화 함수를 사용
    tf.keras.layers.Dropout(0.5), # Dropout 층: 과적합을 방지하기 위해 사용됩
    tf.keras.layers.Dense(len(df['label'].unique()), activation='softmax') # Dense 층: 최종 출력 층으로, 다중 분류를 위한 softmax 활성화 함수사용
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


In [ ]:
# Number of weights in the model
num_weights = sum([tf.size(w).numpy() for w in model.weights])

# Get the training accuracy from the history object
final_training_accuracy = history.history['accuracy'][-1]

num_weights, final_training_accuracy


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test) #테스트 데이터로 모델 성능평가
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


In [ ]:
# 새로운 데이터 넣어서 모델 평가해보기
# 레이블의 인덱스와 실제 레이블 값의 매핑 정보를 포함하는 한글 리스트를 생성
LABELS = ["예/아니오", "대체", "wh- 질문", "금지", "요구사항", "강한 요구사항"]

def predict_intent(sentence):
    # 토큰화 및 패딩
    sequence = tokenizer.texts_to_sequences([sentence])
    padded_sequence = pad_sequences(sequence, maxlen=X.shape[1])

    # 예측
    prediction = model.predict(padded_sequence)
    predicted_label = LABELS[prediction.argmax()]

    return predicted_label

# 예측 테스트
sentence = "필수적으로 진행해 주세요"
print(f"Predicted Intent: {predict_intent(sentence)}")


In [ ]:
# 강한 요구사항을 출력 어려움

In [ ]:
from google.colab import drive
drive.mount('/content/drive')